In [1]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier  # Import Random Forest
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.preprocessing import LabelEncoder
# from sklearn.utils import shuffle
# import re
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# # Download NLTK stopwords data
# # Uncomment the following lines if you haven't downloaded NLTK data
# # import nltk
# # nltk.download('stopwords')
# # nltk.download('punkt')

# # Load dataset
# df = pd.read_csv('./dataset.tsv', delimiter='\t')

# # Drop rows with missing values
# df = df.dropna(subset=['text', 'label'])

# # Shuffle the dataset
# df = shuffle(df, random_state=42)

# # Data preprocessing
# def preprocess_text(text):
#     # Convert to lowercase
#     text = text.lower()
    
#     # Remove special characters, numbers, and extra whitespaces
#     text = re.sub(r'[^a-z\s]', '', text)
    
#     # Tokenization and removing stopwords
#     stop_words = set(stopwords.words('english'))
#     tokens = word_tokenize(text)
#     tokens = [word for word in tokens if word not in stop_words]
    
#     # Join tokens back into text
#     text = ' '.join(tokens)
    
#     return text

# df['text'] = df['text'].apply(preprocess_text)
# label_encoder = LabelEncoder()
# df['label'] = label_encoder.fit_transform(df['label'])

# # Features (X) - 'text', Labels (y) - 'label'
# X = df['text']
# y = df['label']

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # TF-IDF Vectorization
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

# # Random Forest model
# random_forest_model = RandomForestClassifier(n_estimators=100)  # You can adjust the number of trees (n_estimators)
# random_forest_model.fit(X_train_tfidf, y_train)

# # Predictions on the test set
# y_pred = random_forest_model.predict(X_test_tfidf)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")

# # Display classification report for precision, recall, and F1-score
# print("Classification Report:\n", classification_report(y_test, y_pred))

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from joblib import dump

# Uncomment the following lines if you haven't downloaded NLTK data
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

# Load the original dataset
df1 = pd.read_csv('./dataset.tsv', delimiter='\t')

# Load the second dataset
df2 = pd.read_csv('./dark-patterns-v2.csv')

# Check column names
print("Original Dataset Columns:", df1.columns)
print("Second Dataset Columns:", df2.columns)

# Rename columns if necessary
df2 = df2.rename(columns={'Pattern String': 'text'})

# Concatenate the two datasets based on their common columns
df = pd.concat([df1[['text', 'Pattern Category']], df2[['text', 'Pattern Category']]], ignore_index=True)

# Drop rows with missing values
df = df.dropna(subset=['text', 'Pattern Category'])

# Display the count of rows after concatenation
print(df.head())
print(df.count())
# Shuffle the dataset
df = shuffle(df, random_state=42)

# Data preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(tokens)
    return text

df['text'] = df['text'].apply(preprocess_text)
label_encoder = LabelEncoder()
df['Pattern Category'] = label_encoder.fit_transform(df['Pattern Category'])

# Features (X) - 'text', Labels (y) - 'Pattern Category'
X = df['text']
y = df['Pattern Category']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, sublinear_tf=True, smooth_idf=True, norm='l2')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100)  # You can adjust the number of trees (n_estimators)
random_forest_model.fit(X_train_tfidf, y_train)

# Predictions on the test set
y_pred = random_forest_model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report for precision, recall, and F1-score
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the trained model to a file
model_filename = 'random_forest_model.joblib'
dump(random_forest_model, model_filename)

# Save the TF-IDF vectorizer as well for later use in the frontend
vectorizer_filename = 'tfidf_vectorizer.joblib'
dump(tfidf_vectorizer, vectorizer_filename)

# Save the LabelEncoder
label_encoder_filename = 'label_encoder.joblib'
dump(label_encoder, label_encoder_filename)


Original Dataset Columns: Index(['page_id', 'text', 'label', 'Pattern Category'], dtype='object')
Second Dataset Columns: Index(['Pattern String', 'Comment', 'Pattern Category', 'Pattern Type',
       'Where in website?', 'Deceptive?', 'Website Page'],
      dtype='object')
text                3902
Pattern Category    3902
dtype: int64
Accuracy: 0.97
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       0.00      0.00      0.00         4
           2       0.93      0.96      0.95        82
           3       0.97      0.94      0.96       252
           4       1.00      1.00      1.00        21
           5       0.99      1.00      0.99       216
           6       1.00      1.00      1.00         5
           7       0.95      1.00      0.98       106
           8       1.00      0.67      0.80         6
           9       0.96      0.98      0.97        84

    accuracy               

c:\Users\Harsh Goyal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Harsh Goyal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Harsh Goyal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

['label_encoder.joblib']